In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import sklearn.ensemble
import sklearn.metrics
import sklearn.preprocessing
import sklearn.linear_model
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv('features.csv', index_col='match_id')
data.head(5)

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [3]:
missing_data=data.isnull().sum()
missing_data[missing_data>0].sort_values(ascending=False)

first_blood_player2            43987
radiant_flying_courier_time    27479
dire_flying_courier_time       26098
first_blood_player1            19553
first_blood_team               19553
first_blood_time               19553
dire_bottle_time               16143
radiant_bottle_time            15691
radiant_first_ward_time         1836
dire_first_ward_time            1826
radiant_courier_time             692
dire_courier_time                676
dtype: int64

***Пропуск данных***:
***
1. **first_blood_player2**: игры, в которых второй игрок не участвовал в совершении события "первая кровь".
2. **radiant_flying_courier_time, dire_flying_courier_time**: эти значения указывают на то, что никто из игроков соотвествующей(radiant и dire) команды не купил предмет "flying_courier" в первые 5 минут.
3. **first_blood_player1, first_blood_team, first_blood_time**: эти значения одинаковы и указывают на то, что в первые 5 минут вообще не произошло события "первая кровь".
4. **dire_bottle_time, radiant_bottle_time**: эти значения указывают на то, что никто из соотвествующей(dire и radiant) команд не купил предмет "bottle" в первые 5 минут.
5. **radiant_first_ward_time, dire_first_ward_time**: эти значения указывают на то, что никто из игроков соотвествующей(radiant и dire) команды не поставил "наблюдателя" в первые 5 минут.
6. **radiant_courier_time, dire_courier_time**: эти значения указывают на то, что никто из игроков соотвествующей(radiant и dire) команды не купил предмет "courier" в первые 5 минут.
***

***
**Целевой переменной является *radiant_win*, поскольку этот столбец описывает какая команда выиграла.** 
***

In [4]:
#удаляем признаки, связанные с итогами матча и разделяем данные на признаки и целевую переменную
y=data['radiant_win']
X=data.drop(['duration','radiant_win','tower_status_radiant',
             'tower_status_dire','barracks_status_radiant','barracks_status_dire'],axis=1)
#заменяем пустые значения
X=X.fillna(0)

# Градиентный бустинг " в лоб"

In [5]:
kf=sklearn.model_selection.KFold(n_splits=5,random_state=241,shuffle=True)
results=[]

In [6]:
number=[5,10,20,30,40,50,100,200]
for i in number:
    gb=sklearn.ensemble.GradientBoostingClassifier(n_estimators=i,random_state=241)
    print('n_estimators: {:}'.format(i))
    start_time = datetime.datetime.now()
    cr_val=sklearn.model_selection.cross_val_score(gb,X,y,cv=kf,scoring='roc_auc',n_jobs=-1)    
    print ('Time elapsed: {:}'.format(datetime.datetime.now() - start_time))
    print ('Mean score: {:0.3}'.format(cr_val.mean()))
    results.append(cr_val.mean())

n_estimators: 5
Time elapsed: 0:00:18.956424
Mean score: 0.636
n_estimators: 10
Time elapsed: 0:00:28.958082
Mean score: 0.664
n_estimators: 20
Time elapsed: 0:00:52.319463
Mean score: 0.683
n_estimators: 30
Time elapsed: 0:01:14.397130
Mean score: 0.689
n_estimators: 40
Time elapsed: 0:01:36.881645
Mean score: 0.694
n_estimators: 50
Time elapsed: 0:01:59.188998
Mean score: 0.697
n_estimators: 100
Time elapsed: 0:03:49.676233
Mean score: 0.706
n_estimators: 200
Time elapsed: 0:07:31.185642
Mean score: 0.714


***
**Кросс-валидация для градиентного бустинга с 30 деревьями заняла 1 минуту 14 секунд, метрика качества AUC-ROC составила 0.689**
***

***
**При увеличении числа деревьев улучшается точность: так при 200 деревьях она равна 0.714, когда при 30 - всего 0.689. Но при этом сильно увеличивается время обучения. Поэтому необходимо выбирать что важнее: точность или скорость.**
# Меры по ускорению обучения
1. Использовать не всю выборку, а только некоторую её часть.
2. Ограничить глубину деревьев(параметр max_depth).
***

# Логистическая регрессия

In [7]:
data=pd.read_csv('features.csv', index_col='match_id')
#удаляем признаки, связанные с итогами матча и разделяем данные на признаки и целевую переменную
y=data['radiant_win']
X=data.drop(['duration','radiant_win','tower_status_radiant',
             'tower_status_dire','barracks_status_radiant','barracks_status_dire'],axis=1)
#заменяем пустые значения
X=X.fillna(0)
X=sklearn.preprocessing.StandardScaler().fit_transform(X)

# Выбор лучшего параметра регуляризации 

In [8]:
kf=sklearn.model_selection.KFold(n_splits=5,random_state=241,shuffle=True)
arr=np.power(10.0, np.arange(-5, 6))
results=[]
for i in arr:
    model = sklearn.linear_model.LogisticRegression(C=i, random_state=241, n_jobs=-1)
    print('C: {:}'.format(i))
    start_time = datetime.datetime.now()
    cr_val=sklearn.model_selection.cross_val_score(model,X,y,cv=kf,scoring='roc_auc',n_jobs=-1)    
    print ('Time elapsed: {:}'.format(datetime.datetime.now() - start_time))
    print ('Mean score: {:0.3}'.format(cr_val.mean()))
    results.append(cr_val.mean())    

C: 1e-05
Time elapsed: 0:00:05.983838
Mean score: 0.695
C: 0.0001
Time elapsed: 0:00:07.153638
Mean score: 0.711
C: 0.001
Time elapsed: 0:00:10.607039
Mean score: 0.716
C: 0.01
Time elapsed: 0:00:13.812741
Mean score: 0.716
C: 0.1
Time elapsed: 0:00:13.594161
Mean score: 0.716
C: 1.0
Time elapsed: 0:00:13.301381
Mean score: 0.716
C: 10.0
Time elapsed: 0:00:13.410674
Mean score: 0.716
C: 100.0
Time elapsed: 0:00:13.472836
Mean score: 0.716
C: 1000.0
Time elapsed: 0:00:13.705457
Mean score: 0.716
C: 10000.0
Time elapsed: 0:00:14.356586
Mean score: 0.716
C: 100000.0
Time elapsed: 0:00:13.444115
Mean score: 0.716


In [9]:
print ('Best mean score: : {:}'.format(max(results)))
print ('Best C: {:0.3}'.format(arr[results.index(max(results))]))

Best mean score: : 0.716341462186996
Best C: 0.01


***
**Лучший результат логистической регресии равен 0.716, что примерно равно результату при градиентном бустинге с 200 деревьями. При этом скорость логистической регресии намного выше: 14 секунд против 7 минут и 31 секунды.**
***

# Удаление категориальных признаков

In [10]:
X=data.drop(['duration','radiant_win','tower_status_radiant',
             'tower_status_dire','barracks_status_radiant','barracks_status_dire'],axis=1)
X=X.fillna(0)
X=X.drop('lobby_type',axis=1)
for i in range(1,6):
    X=X.drop('r'+str(i)+'_hero',axis=1)
    X=X.drop('d'+str(i)+'_hero',axis=1)
X=sklearn.preprocessing.StandardScaler().fit_transform(X)    

In [11]:
cat_results=[]
for i in arr:
    model = sklearn.linear_model.LogisticRegression(C=i, random_state=241, n_jobs=-1)
    print('C: {:}'.format(i))
    start_time = datetime.datetime.now()
    cr_val=sklearn.model_selection.cross_val_score(model,X,y,cv=kf,scoring='roc_auc',n_jobs=-1)    
    print ('Time elapsed: {:}'.format(datetime.datetime.now() - start_time))
    print ('Mean score: {:0.3}'.format(cr_val.mean()))
    cat_results.append(cr_val.mean())   
    

C: 1e-05
Time elapsed: 0:00:05.762536
Mean score: 0.695
C: 0.0001
Time elapsed: 0:00:06.757977
Mean score: 0.711
C: 0.001
Time elapsed: 0:00:09.886298
Mean score: 0.716
C: 0.01
Time elapsed: 0:00:12.974511
Mean score: 0.716
C: 0.1
Time elapsed: 0:00:12.646639
Mean score: 0.716
C: 1.0
Time elapsed: 0:00:12.696773
Mean score: 0.716
C: 10.0
Time elapsed: 0:00:12.729861
Mean score: 0.716
C: 100.0
Time elapsed: 0:00:12.906331
Mean score: 0.716
C: 1000.0
Time elapsed: 0:00:13.332465
Mean score: 0.716
C: 10000.0
Time elapsed: 0:00:13.308399
Mean score: 0.716
C: 100000.0
Time elapsed: 0:00:12.704793
Mean score: 0.716


In [12]:
print ('Best mean score: : {:}'.format(max(cat_results)))
print ('Best C: {:0.3}'.format(arr[cat_results.index(max(cat_results))]))

Best mean score: : 0.7164009506527343
Best C: 0.01


***
**Полученная метрика качества не отличается от предыдущей и равна 0.716, совпадает также и оптимальный параметр регуляризации С=0.01. То, что результат не изменился можно обьяснить тем, что алгоритмы библиотеки sklearn хорошо оптимизированы.**
***

# "Мешок слов"

In [13]:
#Вычисление количества героев
X=data.drop(['duration','radiant_win','tower_status_radiant',
             'tower_status_dire','barracks_status_radiant','barracks_status_dire'],axis=1)
X=X.fillna(0)
N=X['r5_hero'].value_counts().sort_index(ascending=False).head(1)
print("Number of heroes: {:}".format(N))

Number of heroes: 112    3851
Name: r5_hero, dtype: int64


In [14]:
X_pick = np.zeros((X.shape[0], 112))
for i, match_id in enumerate(X.index):
    for p in range(1,5):
        X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1  

In [15]:
X_h=pd.DataFrame(X_pick, index=X.index)
for i in range(1,6):
    X=X.drop('r'+str(i)+'_hero',axis=1)
    X=X.drop('d'+str(i)+'_hero',axis=1)
X=pd.DataFrame(sklearn.preprocessing.StandardScaler().fit_transform(X),index=X.index)
X=pd.concat([X, X_h], axis=1)

In [16]:
bag_results=[]
for i in arr:
    model = sklearn.linear_model.LogisticRegression(C=i, random_state=241, n_jobs=-1)
    print('C: {:}'.format(i))
    start_time = datetime.datetime.now()
    cr_val=sklearn.model_selection.cross_val_score(model,X,y,cv=kf,scoring='roc_auc',n_jobs=-1)    
    print ('Time elapsed: {:}'.format(datetime.datetime.now() - start_time))
    print ('Mean score: {:0.3}'.format(cr_val.mean()))
    bag_results.append(cr_val.mean())   

C: 1e-05
Time elapsed: 0:00:07.096051
Mean score: 0.698
C: 0.0001
Time elapsed: 0:00:08.172603
Mean score: 0.72
C: 0.001
Time elapsed: 0:00:13.115889
Mean score: 0.738
C: 0.01
Time elapsed: 0:00:19.757861
Mean score: 0.743
C: 0.1
Time elapsed: 0:00:26.401189
Mean score: 0.743
C: 1.0
Time elapsed: 0:00:29.208753
Mean score: 0.743
C: 10.0
Time elapsed: 0:00:29.211702
Mean score: 0.743
C: 100.0
Time elapsed: 0:00:28.408170
Mean score: 0.743
C: 1000.0
Time elapsed: 0:00:29.160379
Mean score: 0.743
C: 10000.0
Time elapsed: 0:00:30.440767
Mean score: 0.743
C: 100000.0
Time elapsed: 0:00:30.102089
Mean score: 0.743


In [17]:
print ('Best mean score: : {:}'.format(max(bag_results)))
print ('Best C: {:0.3}'.format(arr[bag_results.index(max(bag_results))]))

Best mean score: : 0.7432240032912947
Best C: 0.1


***
**Использование "мешка слов" улучшило показатели: теперь лучший показатель равен 0.743. При этом C=0.1**
**Выбор героя игроком можно отнести к важным признакам, но из-за того, что эти признаки были категориальные они не влияли на точность предсказания. Использование метода "мешок слов" позволило заменить этот шум на осмысленную разреженную матрицу, которую легко использовать с логистической регрессией**
***

# Тестовая выборка

In [18]:
#Обучение
model=sklearn.linear_model.LogisticRegression(C=0.1, random_state=241, n_jobs=-1)
model.fit(X,y)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=241, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [19]:
X_test=pd.read_csv('features_test.csv',index_col='match_id')
X_test=X_test.fillna(0)

X_tpick = np.zeros((X_test.shape[0], 112))
for i, match_id in enumerate(X_test.index):
    for p in range(1,5):
        X_tpick[i, X_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_tpick[i, X_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1  

X_h=pd.DataFrame(X_tpick, index=X_test.index)
for i in range(1,6):
    X_test=X_test.drop('r'+str(i)+'_hero',axis=1)
    X_test=X_test.drop('d'+str(i)+'_hero',axis=1)
X_test=pd.DataFrame(sklearn.preprocessing.StandardScaler().fit_transform(X_test),index=X_test.index)
X_test=pd.concat([X_test, X_h], axis=1)

In [20]:
y_test = model.predict_proba(X_test)[:, 1]
y_test=pd.DataFrame({'radiant_win': y_test}, index=X_test.index)
y_test.index.name='match_id'
print(y_test.sort_values(by='radiant_win',ascending=False).head(1))
print(y_test.sort_values(by='radiant_win').head(1))

          radiant_win
match_id             
14176        0.995255
          radiant_win
match_id             
33469        0.007968


**Команда radiant имела наибольшый шанс(равен 0.9953) выиграть в матче 14176 и наименьший(равен 0.00797) в матче 33469**